## Plotting all publicly-owned existing airports and helipads in Chicago Metropolitan Area

The purpose of this script is to plot the (lat, lon) of existing airport and helipad infrastructures of Chicago region 

To install fiona and GDAL, watch this: https://www.youtube.com/watch?v=GRyBR--2zFo
https://www.lfd.uci.edu/~gohlke/pythonlibs/#gdal

If geos_c.dll not found, go to here an download it: https://www.dll-files.com/geos_c.dll.html
Place it in the following path: Anaconda3\Library\bin\geos_c.dll


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
import contextily as ctx
import tilemapbase
import mplleaflet

In [ ]:
tilemapbase.start_logging()

tilemapbase.init(create=True)

t = tilemapbase.tiles.build_OSM()

LatLim = [41, 42.5]
LonLim = [-89, -87]

In [ ]:
extent = tilemapbase.Extent.from_lonlat(min(LonLim), max(LonLim), min(LatLim), max(LatLim))

In [ ]:
from shapely import speedups
speedups.disable()

In [ ]:
street_map = gpd.read_file("./Datasets/ChicagoData2/subzones17.shp")
street_map.to_crs(epsg=4326, inplace=True) # this converts to lat, long axis, #3857 or 4326

major = pd.read_excel("Datasets/Airports_Helipads_Chicago_Area_PU.xlsx", sheet_name = "Major_Airports_Chicago_Area")
regional = pd.read_excel("Datasets/Airports_Helipads_Chicago_Area_PU.xlsx", sheet_name = "Regional_Airports_Chicago_Area")
heliport = pd.read_excel("Datasets/Airports_Helipads_Chicago_Area_PU.xlsx", sheet_name = "Heliports_Chicago_Area")

Lat_regional = regional["LatD"]
Lon_regional = regional["LonD"]

Lat_heliports = heliport["LatD"]
Lon_heliports = heliport["LonD"]

Lat_major = major["LatD"]
Lon_major = major["LonD"]

d = {"LatD" : Lat_regional, "LonD" : Lon_regional}
all_p = pd.DataFrame(data=d)
lat_m_regional  = []
lon_m_regional = []
for i in range(len(all_p)):
    latt, lonn = merc((all_p["LatD"][i],all_p["LonD"][i]))
    lat_m_regional.append(latt)    
    lon_m_regional.append(lonn)
    #print(int(latt))
    #all_p['coords_x'] = all_p['LonD'].apply(lambda x: merc((all_p["LatD"][i],all_p["LonD"][i]))[0])
    #all_p['coords_y'] = all_p['LonD'].apply(lambda x: merc((all_p["LatD"][i],all_p["LonD"][i]))[1])

d = {"LatD" : Lat_heliports, "LonD" : Lon_heliports}
all_p = pd.DataFrame(data=d)
lat_m_heliports  = []
lon_m_heliports = []
for i in range(len(all_p)):
    latt, lonn = merc((all_p["LatD"][i],all_p["LonD"][i]))
    lat_m_heliports.append(latt)    
    lon_m_heliports.append(lonn)    

d = {"LatD" : Lat_major, "LonD" : Lon_major}
all_p = pd.DataFrame(data=d)
lat_m_major  = []
lon_m_major = []
for i in range(len(all_p)):
    latt, lonn = merc((all_p["LatD"][i],all_p["LonD"][i]))
    lat_m_major.append(latt)    
    lon_m_major.append(lonn)

    
fig, ax = plt.subplots(figsize = (15, 15), dpi=600)
street_map.plot(ax=ax, alpha=0.3, color="grey")
#plotter = tilemapbase.Plotter(extent, tilemapbase.tiles.build_OSM(), width=600)
#plotter.plot(ax)
#plt.show()
plt.scatter(Lon_regional, Lat_regional, color="red")
plt.scatter(Lon_heliports, Lat_heliports, color="blue")
plt.scatter(Lon_major, Lat_major, color="orange")

plt.xlim([-89, -87])
plt.ylim([41,42.5])
plt.show()

In [ ]:
import pandas as pd
import geopandas
import geopandas.datasets

In [ ]:
# obtian the sum of confirmed cases and fatalities in different boroughs
# load the data from geopandas.datasets
nyc_shp = geopandas.read_file(geopandas.datasets.get_path('nybb'))
# create a dataframe
con_fa_nyc=pd.DataFrame()
con_fa_nyc['longitude']=[985000,1000000,970000 ,1040000,925000]
con_fa_nyc['latitude']=[180000,250000,220000,200000,150000]
# merge con_fa_nyc and nyc_shp
nyc_shp.head()

In [ ]:
nyc_shp_1 = nyc_shp.to_crs(epsg=3857)
ax = nyc_shp_1.plot(figsize=(10, 10), alpha=0.5, edgecolor='k' ,cmap='Reds',legend=True,scheme="quantiles")
#Add background tiles to plot
ctx.add_basemap(ax)

In [ ]:
import math
from ast import literal_eval
def merc(Coords):
    """
    Converts to Mercator Coordinates from Latitude, Longitude in degrees because bokeh module depends on it
    Input = e.g., (42, -88)
    Output = e.g., (-9780000, -9745000)
    """
    Coordinates = Coords
    
    lat = Coordinates[0]
    lon = Coordinates[1]
    
    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    return (x, y)

In [ ]:
import numpy as np
def TripDef(LatD, LonD, LatA, LonA):
    """
    TripDef function defines a UAM trip from Departure Aerodrome to Arrival Aerodrome
    
    Input: 
        LatD: [Type: float] : Latitude of Departure UAM Aerodrome
        LonD: [Type: float] : Longitude of Departure UAM Aerodrome
        LatA: [Type: float] : Latitude of Arrival UAM Aerodrome
        LonA: [Type: float] : Longitude of Arrival UAM Aerodrome
    
    Output:
        latitude position, longitude position
        
    """
    
    # Find the slope of the trip, with longitudinal axis being the x-axis and latitude axis being the y-axis
    delta = (LatA-LatD)/(LonA-LonD)

    # Use the linear function equation to define the time steps
    Lon = np.linspace(LonD, LonA, 100)    
    Lat = (delta * (Lon - LonD) + LatD)
    return Lon, Lat

In [ ]:
import math

def distance_on_unit_sphere(lat1, long1, lat2, long2):

    # Convert latitude and longitude to
    # spherical coordinates in radians.
    degrees_to_radians = math.pi/180.0

    # phi = 90 - latitude
    phi1 = (90.0 - lat1)*degrees_to_radians
    phi2 = (90.0 - lat2)*degrees_to_radians

    # theta = longitude
    theta1 = long1*degrees_to_radians
    theta2 = long2*degrees_to_radians

    # Compute spherical distance from spherical coordinates.

    # For two locations in spherical coordinates
    # (1, theta, phi) and (1, theta', phi')
    # cosine( arc length ) =
    # sin phi sin phi' cos(theta-theta') + cos phi cos phi'
    # distance = rho * arc length

    cos = (math.sin(phi1)*math.sin(phi2)*math.cos(theta1 - theta2) +
    math.cos(phi1)*math.cos(phi2))
    arc = math.acos( cos )

    # Remember to multiply arc by the radius of the earth
    # in your favorite set of units to get length.
    return arc*6371*0.621371

In [ ]:
error = 10
step = 0.01
Lat1 = Lat_regional[7]
Lat2 = Lat1
Lon1 = Lon_regional[7]
Lon2 = Lon1
desired_distance = 4.28
while error > 1:
    dist_mi = distance_on_unit_sphere(Lat1,Lon1,Lat2,Lon2)
    error = abs(dist_mi - desired_distance)
    Lon2 += step

x, y = merc((Lat2, Lon2))
u,v = merc((Lat1, Lon1))

width = x-u
#print(x-u, y-v)

In [ ]:
"""
Ref 1: https://towardsdatascience.com/exploring-and-visualizing-chicago-transit-data-using-pandas-and-bokeh-part-ii-intro-to-bokeh-5dca6c5ced10
Ref 2: https://stackoverflow.com/questions/57051517/cant-plot-dots-over-tile-on-bokeh
Ref 3: https://docs.bokeh.org/en/latest/docs/reference/tile_providers.html
Ref 4: https://matplotlib.org/2.0.2/examples/color/named_colors.html
Ref 5: https://stackoverflow.com/questions/44281863/saving-html-file-with-images-in-bokeh/44282125
"""
from bokeh.plotting import figure, show, output_notebook, output_file, save
from bokeh.tile_providers import CARTODBPOSITRON, ESRI_IMAGERY, OSM, STAMEN_TERRAIN, STAMEN_TERRAIN_RETINA
from bokeh.tile_providers import get_provider, Vendors 

tile_provider = get_provider(Vendors.ESRI_IMAGERY)

p = figure(x_range=(-9780000-100000, -9745000+100000), y_range=(5130000, 5160000),x_axis_type="mercator", y_axis_type="mercator", title="Chicago Metropolitan Area", plot_width=1875, plot_height=910)
p.add_tile(tile_provider)
p.circle(lat_m_regional, lon_m_regional, color = 'white', alpha = 1, size = 10)
p.circle(lat_m_heliports, lon_m_heliports, color = 'deepskyblue', alpha = 1, size = 10)
p.circle(lat_m_major, lon_m_major, color = 'yellow', alpha = 1, size = 10)

# plot the trip, feel free to change the origin and destination coordinates
idxD = 7
idxA = 6

LatD = lat_m_regional[idxD]
LonD = lon_m_regional[idxD]
LatA = lat_m_heliports[idxA]
LonA = lon_m_heliports[idxA]

LonTrip, LatTrip = TripDef(LatD, LonD, LatA, LonA)
p.line(LatTrip, LonTrip, color = 'white', line_width = 2)

# Don't forget to change the origin and destination here too!
tripDistGeodesic = distance_on_unit_sphere(Lat_regional[idxD],Lon_regional[idxD],Lat_heliports[idxA],Lon_heliports[idxA])
print(f'The trip distance is {tripDistGeodesic} miles')

# plot the reachable radius along the trip
for j in range(len(LatTrip)):
    if j % 10 == 0:
        p.circle(LatTrip[j], LonTrip[j], color = 'yellow', alpha = 1, size = 1)
        p.ellipse(LatTrip[j],LonTrip[j], alpha = 0.3, width=width, height=width, color="white")

output_notebook()
show(p)
output_file("test.html")
save(p)

In [4]:
import pandas as pd

class LoadExistingUAMaerodromeInfrastructure:
    def __init__(self, MetroName):
        self.MetroName = MetroName
        self.FilePath = "Datasets/" + self.MetroName + "/" + self.MetroName + ".xlsx"
        
        self.lat_regional_merc, self.lon_regional_merc = self.ReadExcelSheet(self.FilePath, "Regional")
        self.lat_major_merc, self.lon_major_merc = self.ReadExcelSheet(self.FilePath, "Major")
        self.lat_heliports_merc, self.lon_heliports_merc = self.ReadExcelSheet(self.FilePath, "Heliports")
        
        self.lat_regional_deg, self.lon_regional_deg = self.ReadExcelSheetInDegreesOnly(self.FilePath, "Regional")
        self.lat_major_deg, self.lon_major_deg = self.ReadExcelSheetInDegreesOnly(self.FilePath, "Major")
        self.lat_heliports_deg, self.lon_heliports_deg = self.ReadExcelSheetInDegreesOnly(self.FilePath, "Heliports")
        
    def ReadExcelSheet(self, FilePath, SheetName):
        loadSheet = pd.read_excel(FilePath, sheet_name = SheetName)
        lat_deg = loadSheet["LatD"]
        lon_deg = loadSheet["LonD"]
        lat_merc, lon_merc = self.InMercator(lat_deg, lon_deg)
        return lat_merc, lon_merc
    
    def ReadExcelSheetInDegreesOnly(self, FilePath, SheetName):
        loadSheet = pd.read_excel(FilePath, sheet_name = SheetName)
        lat_deg = loadSheet["LatD"]
        lon_deg = loadSheet["LonD"]
        return lat_deg, lon_deg
        
    def InMercator(self, lat_deg, lon_deg):
        d = {"LatD" : lat_deg, "LonD" : lon_deg}
        all_p = pd.DataFrame(data = d)
        
        lat_merc = []
        lon_merc = []
        for i in range(len(all_p)):
            lat, lon = self.Degrees2Mercator((all_p["LatD"][i], all_p["LonD"][i]))
            lat_merc.append(lat)
            lon_merc.append(lon)
        
        return lat_merc, lon_merc
    
    def Degrees2Mercator(self, Coords):
        """
        Converts to Mercator Coordinates from Latitude, Longitude in degrees because bokeh module depends on it
        Input = e.g., (42, -88)
        Output = e.g., (-9780000, -9745000)
        """
        Coordinates = Coords
    
        lat = Coordinates[0]
        lon = Coordinates[1]
    
        r_major = 6378137.000
        x = r_major * math.radians(lon)
        scale = x/lon
        y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + lat * (math.pi/180.0)/2.0)) * scale
    
        return (x, y)
    
    def __call__(self):
        print(self.lat_regional_merc, self.lon_regional_merc, self.lat_major_merc, self.lon_major_merc, self.lat_heliports_merc, self.lon_heliports_merc)
        return self.lat_regional_merc, self.lon_regional_merc, self.lat_major_merc, self.lon_major_merc, self.lat_heliports_merc, self.lon_heliports_merc
    

In [9]:
"""
Ref 1: https://towardsdatascience.com/exploring-and-visualizing-chicago-transit-data-using-pandas-and-bokeh-part-ii-intro-to-bokeh-5dca6c5ced10
Ref 2: https://stackoverflow.com/questions/57051517/cant-plot-dots-over-tile-on-bokeh
Ref 3: https://docs.bokeh.org/en/latest/docs/reference/tile_providers.html
Ref 4: https://matplotlib.org/2.0.2/examples/color/named_colors.html
Ref 5: https://stackoverflow.com/questions/44281863/saving-html-file-with-images-in-bokeh/44282125
"""
import math
import numpy as np
from bokeh.plotting import figure, show, output_notebook, output_file, save
from bokeh.tile_providers import CARTODBPOSITRON, ESRI_IMAGERY, OSM, STAMEN_TERRAIN, STAMEN_TERRAIN_RETINA
from bokeh.tile_providers import get_provider, Vendors 

class TripMapper(LoadExistingUAMaerodromeInfrastructure):
    def __init__(self, MetroName):
        LoadExistingUAMaerodromeInfrastructure.__init__(self, MetroName)
        self.p = self.MapperInfrastructure()
    
    def MapperInfrastructure(self):
        tile_provider = get_provider(Vendors.ESRI_IMAGERY)
        p = figure(x_range=(-9780000-100000, -9745000+100000), y_range=(5130000, 5160000),x_axis_type="mercator", y_axis_type="mercator", title="Chicago Metropolitan Area", plot_width=1875, plot_height=910)
        p.add_tile(tile_provider)
        p.circle(self.lat_regional_merc, self.lon_regional_merc, color = 'white', alpha = 1, size = 10)
        p.circle(self.lat_major_merc, self.lon_major_merc, color = 'deepskyblue', alpha = 1, size = 10)
        p.circle(self.lat_heliports_merc, self.lon_heliports_merc, color = 'yellow', alpha = 1, size = 10)
        return p
    
    def DrawGeodesicTrip(self, idxDep, idxArr):
        latDep_deg = self.lat_regional_deg[idxDep]
        lonDep_deg = self.lon_regional_deg[idxDep]
        latArr_deg = self.lat_heliports_deg[idxArr]
        lonArr_deg = self.lon_heliports_deg[idxArr]
        
        geodesicDistance = self.GeodesicDistance(self.lat_regional_deg[idxDep], self.lon_regional_deg[idxDep], self.lat_heliports_deg[idxArr], self.lon_heliports_deg[idxArr])
        print(f'Geodesic Trip Distance between {idxDep} - {idxArr}: {geodesicDistance} miles')
        
        latDep_merc = self.lat_regional_merc[idxDep]
        lonDep_merc = self.lon_regional_merc[idxDep]
        latArr_merc = self.lat_heliports_merc[idxArr]
        lonArr_merc = self.lon_heliports_merc[idxArr]
                
        # Find the slope of the trip, with longitudinal axis being the x-axis and latitude axis being the y-axis
        delta = (self.lat_heliports_merc[idxArr]-self.lat_regional_merc[idxDep])/(self.lon_heliports_merc[idxArr]-self.lon_regional_merc[idxDep])
        
        # Use the linear function equation to define the time steps
        Lon = np.linspace(self.lon_regional_merc[idxDep], self.lon_heliports_merc[idxArr], 100)    
        Lat = (delta * (Lon - self.lon_regional_merc[idxDep]) + self.lat_regional_merc[idxDep])
        self.p.line(Lat, Lon, color = 'white', line_width = 2)

        return None
    
    def ShowMap(self): 
        output_notebook()
        show(self.p)
        output_file("test.html")
        save(self.p)
        
    def GeodesicDistance(self, lat1, long1, lat2, long2):
        # Convert latitude and longitude to
        # spherical coordinates in radians.
        degrees_to_radians = math.pi/180.0

        # phi = 90 - latitude
        phi1 = (90.0 - lat1)*degrees_to_radians
        phi2 = (90.0 - lat2)*degrees_to_radians

        # theta = longitude
        theta1 = long1*degrees_to_radians
        theta2 = long2*degrees_to_radians

        # Compute spherical distance from spherical coordinates.

        # For two locations in spherical coordinates
        # (1, theta, phi) and (1, theta', phi')
        # cosine( arc length ) =
        # sin phi sin phi' cos(theta-theta') + cos phi cos phi'
        # distance = rho * arc length

        cos = (math.sin(phi1)*math.sin(phi2)*math.cos(theta1 - theta2) +
        math.cos(phi1)*math.cos(phi2))
        arc = math.acos( cos )

        # Remember to multiply arc by the radius of the earth
        # in your favorite set of units to get length.
        return arc*6371*0.621371
    
    def __call__(self):
        print(f'Geodesic Trip Distance: {self.geodesicDistance} miles')
        
        

In [12]:
# This is the main code that creates instances
Chicago = TripMapper("Chicago")
for dest in range(8,10):
    for arr in range(7,13):
        Chicago.DrawGeodesicTrip(dest,arr)
Chicago.ShowMap()


Geodesic Trip Distance between 8 - 7: 26.766534861295327 miles
Geodesic Trip Distance between 8 - 8: 26.40956334288139 miles
Geodesic Trip Distance between 8 - 9: 23.477092855538142 miles
Geodesic Trip Distance between 8 - 10: 24.339546803620003 miles
Geodesic Trip Distance between 8 - 11: 34.23910690628202 miles
Geodesic Trip Distance between 8 - 12: 37.78903392359978 miles
Geodesic Trip Distance between 9 - 7: 27.989172047130733 miles
Geodesic Trip Distance between 9 - 8: 23.731837095288466 miles
Geodesic Trip Distance between 9 - 9: 9.212262688107067 miles
Geodesic Trip Distance between 9 - 10: 4.879485500283269 miles
Geodesic Trip Distance between 9 - 11: 22.81087555881577 miles
Geodesic Trip Distance between 9 - 12: 20.36758411829005 miles


Loading BokehJS ...

In [8]:
lat_reg, lon_reg, lat_major, lon_major, lat_heli, lon_heli = LoadExistingUAMaerodromeInfrastructure("Chicago")()

[-9874794.108278524, -9867493.222569529, -9850145.703334928, -9842740.931805834, -9841486.849714708, -9840116.713959744, -9832839.41252052, -9824339.863269854, -9810549.518166084, -9808136.457931753, -9807346.949178493, -9804992.609975556, -9789627.143558575, -9785700.299039328, -9782268.00373666, -9761478.603355585, -9744674.453112502, -9741311.56241766, -9733370.054679658, -9731373.4901419, -9730249.9579836, -9714175.386399133, -9705000.313375548, -9687010.190012747] [5150145.254341048, 5221157.873460321, 5126611.418251442, 5076315.392817967, 5182897.625554424, 5253061.520044874, 5191868.841193519, 5145118.874834931, 5116223.935723929, 5159586.908583733, 5101134.041745811, 5209424.433936954, 5250226.142933424, 5177542.709617124, 5223477.119713797, 5068166.444498375, 5092305.518608919, 5046965.118711315, 4976824.784107054, 5103076.993441208, 5089149.89780862, 5095323.862256368, 5005473.083555919, 5079303.357393929] [-9785517.62103874, -9768552.8467892] [5157774.299280344, 5128976.8852